<a href="https://colab.research.google.com/github/econdatatech/AIML427/blob/main/Part%201%20draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

tar: spark-3.0.0-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
!pip3 uninstall -y pyspark
!pip3 install pyspark==3.0.2


Found existing installation: pyspark 3.0.2
Uninstalling pyspark-3.0.2:
  Successfully uninstalled pyspark-3.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyspark-3.0.2-py2.py3-none-any.whl


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Test the spark
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3, False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

#amended from https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

# Load the data stored in csv format as a DataFrame.
data = spark.read.load("kdd.data",
                     format="csv", sep=",", inferSchema="true", header="false").toDF('duration', 
                      'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
                      'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
                      'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 
                      'is_host_login', 'is_guest_login','count', 'srv_count', 'serror_rate', 'srv_serror_rate', 
                      'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
                      'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                      'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                      'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate','label')

assembler = VectorAssembler().setInputCols(['duration', 
                      'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
                      'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
                      'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 
                      'is_host_login', 'is_guest_login','count', 'srv_count', 'serror_rate', 'srv_serror_rate', 
                      'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
                      'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                      'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                      'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']).setOutputCol('features')

data=assembler.transform(data)
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
#featureIndexer = [VectorIndexer(inputCol=column, outputCol=column+"_index", maxCategories=4).fit(data) for column in list(set(data.columns[:-1])) ]
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)


# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3],2342)

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions_train = model.transform(trainingData)

predictions_test = model.transform(testData)

# Select example rows to display.
# predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy_train = evaluator.evaluate(predictions_train)
print("Train accuracy = %g " % (accuracy_train))

accuracy_test = evaluator.evaluate(predictions_test)
print("Test accuracy = %g " % (accuracy_test))


#treeModel = model.stages[2]
# summary only
#print(treeModel)


Report the training and test results including the max, min,
average accuracy and the standard deviation obtained from the 10 runs,
and the running time of each program.

Train accuracy = 0.943957 
Test accuracy = 0.941803 
DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5325828822a8, depth=5, numNodes=47, numClasses=2, numFeatures=41


In [ ]:
data.show()


+----+----+----+----+----+-----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+--------------------+
|fea1|fea2|fea3|fea4|fea5| fea6|fea7|fea8|fea9|fea10|fea11|fea12|fea13|fea14|fea15|fea16|fea17|fea18|fea19|fea20|fea21|fea22|fea23|fea24|fea25|fea26|fea27|fea28|fea29|fea30|fea31|fea32|fea33|fea34|fea35|fea36|fea37|fea38|fea39|fea40|fea41|  label|            features|
+----+----+----+----+----+-----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+--------------------+
|   0|   0|  17|   9| 491|    0|   0|   0|   0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    2|    2|  0.0|  0.0|  0.0|  0.0|  1.0|  0.0|  0.0|  150|   25| 0